# Climate Data from Home System

In [124]:
from erddapy import ERDDAP
import pandas as pd
import datetime

# for secondary/derived parameters
from metpy.units import units
import metpy.calc as mpcalc

In [125]:
server_url = 'http://raspberrypi.local:8080/erddap'

In [126]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for='MoonFlower'))
print(df['Dataset ID'].values)

['tempest_moonflower_wx' 'channel_1027974_thingspeak'
 'channel_1037066_thingspeak' 'channel_1047747_thingspeak'
 'channel_843357_thingspeak' 'allDatasets']


In [127]:
dataset_id='tempest_moonflower_wx'

try:
    d = ERDDAP(server=server_url,
        protocol='tabledap',
        response='csv'
    )
    d.dataset_id=dataset_id
except HTTPError:
    print('Failed to generate url {}'.format(dataset_id))

try:
    df_m = d.to_pandas(
                index_col='time (UTC)',
                parse_dates=True,
                skiprows=(1,)  # units information can be dropped.
                )
    df_m.sort_index(inplace=True)
    df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

except:
    print(f"something failed in data download {dataset_id}")
    pass

df_m.drop(columns=['device_id', 'bucket_step_minutes', 'wind_lull','wind_interval'],inplace=True)
#stats are all utc driven - but we really want local daily values
df_m=df_m.tz_convert('US/Pacific')

# calculations of various parameters... metpy?
# HDD/CDD, dewpointTemp

In [128]:
df_m['dewpointTemp']=mpcalc.dewpoint_from_relative_humidity(df_m.temperature.values * units.degC,
                                                            df_m.humidity.values * units.percent)
#wetbulb from metpy had issues

In [129]:
df_m['SLP']=df_m.statioinpressure.values * (1+((1013.25/df_m.statioinpressure.values)**((287.05*0.0065)/9.80665)) * (0.0065*87.3)/288.15)**(9.80665/(287.05*0.0065))

In [131]:
df_daily_max = df_m.resample('D').max()
df_daily_min = df_m.resample('D').min()
df_daily_ave = df_m.resample('D').mean()
df_daily_total = df_m.resample('1T').mean().resample('D').sum()
df_m.sample()

,type,wind_avg,wind_gust,wind_dir,statioinpressure,temperature,humidity,lux,uv,solar_radiation,...,strike_distance,strike_count,battery,report_interval,local_daily_precip,precip_final,local_daily_precip_final,precip_analysis_type,dewpointTemp,SLP
time (UTC),,,,,,,,,,,,,,,,,,,,,
2020-06-26 16:50:00-07:00,obs_st,0.19,0.94,53.0,1008.2,19.4,71.0,35045,1.59,292.0,...,0.0,0.0,2.7,1.0,0.0,0.0,0.0,1.0,14.01111,1018.689144


In [146]:
use_current_month = True

if use_current_month:
    current_month = datetime.datetime.now().month
else:
    current_month = 6

current_month_grid_data=pd.DataFrame()
current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
current_month_grid_data = pd.concat([current_month_grid_data,
                                     df_daily_min[df_daily_min.index.month==current_month].temperature,
                                     df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].SLP.round(1),
                                     df_daily_total[df_daily_total.index.month==current_month].solar_radiation,
                                     df_daily_max[df_daily_max.index.month==current_month].uv.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_avg.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_dir.astype(int),
                                     df_daily_max[df_daily_max.index.month==current_month].wind_gust.round(1)
                                    ],axis=1)
current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature','mean SLP','total_solar_radiation','max_uv_index','average speed','average dir','max gust')

In [147]:
#this should go to erddap
current_month_grid_data.to_csv(f'MoonflowerTempest_2020{str(current_month).zfill(2)}.csv')

In [134]:
#monthly stats
current_month_grid_data.describe()

,max_temperature,min_temperature,mean_temperature,mean SLP,total_solar_radiation,max_uv_index,average speed,average dir,max gust
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,21.685714,10.466667,15.257143,1019.223810,338803.476190,1.442857,0.428571,86.285714,5.723810
std,3.437337,1.612865,1.780610,3.198266,134018.014208,0.586150,0.100712,21.485211,1.380545
min,15.200000,8.100000,12.200000,1011.500000,102809.000000,0.400000,0.300000,60.000000,3.800000
25%,18.800000,9.100000,14.100000,1017.300000,208604.000000,0.900000,0.400000,72.000000,4.700000
50%,22.300000,10.600000,15.000000,1019.900000,320131.000000,1.500000,0.400000,77.000000,5.400000
75%,24.400000,11.100000,16.600000,1021.100000,466109.000000,1.800000,0.500000,94.000000,6.500000
max,28.200000,13.600000,18.600000,1024.300000,508393.000000,2.600000,0.700000,149.000000,8.600000


In [135]:
### need to manage daily records, monthly records, alltime records

In [136]:
df=df_m.tz_convert('US/Pacific')

In [137]:
df.head()

,type,wind_avg,wind_gust,wind_dir,statioinpressure,temperature,humidity,lux,uv,solar_radiation,...,strike_distance,strike_count,battery,report_interval,local_daily_precip,precip_final,local_daily_precip_final,precip_analysis_type,dewpointTemp,SLP
time (UTC),,,,,,,,,,,,,,,,,,,,,
2020-06-25 21:04:00-07:00,obs_st,0.96,2.37,82.0,1008.9,16.2,82.0,813,0.04,7.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,13.126095,1019.395035
2020-06-25 21:05:00-07:00,obs_st,0.59,1.88,154.0,1009.0,16.2,81.0,765,0.04,6.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,12.938425,1019.495876
2020-06-25 21:06:00-07:00,obs_st,0.21,0.89,147.0,1009.0,16.1,81.0,719,0.03,6.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,12.840920,1019.495876
2020-06-25 21:07:00-07:00,obs_st,0.43,1.43,128.0,1008.9,16.1,82.0,676,0.03,6.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,13.028448,1019.395035
2020-06-25 21:08:00-07:00,obs_st,0.17,0.72,127.0,1009.0,16.1,81.0,635,0.03,5.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,12.840920,1019.495876


In [138]:
df_m.head()

,type,wind_avg,wind_gust,wind_dir,statioinpressure,temperature,humidity,lux,uv,solar_radiation,...,strike_distance,strike_count,battery,report_interval,local_daily_precip,precip_final,local_daily_precip_final,precip_analysis_type,dewpointTemp,SLP
time (UTC),,,,,,,,,,,,,,,,,,,,,
2020-06-25 21:04:00-07:00,obs_st,0.96,2.37,82.0,1008.9,16.2,82.0,813,0.04,7.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,13.126095,1019.395035
2020-06-25 21:05:00-07:00,obs_st,0.59,1.88,154.0,1009.0,16.2,81.0,765,0.04,6.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,12.938425,1019.495876
2020-06-25 21:06:00-07:00,obs_st,0.21,0.89,147.0,1009.0,16.1,81.0,719,0.03,6.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,12.840920,1019.495876
2020-06-25 21:07:00-07:00,obs_st,0.43,1.43,128.0,1008.9,16.1,82.0,676,0.03,6.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,13.028448,1019.395035
2020-06-25 21:08:00-07:00,obs_st,0.17,0.72,127.0,1009.0,16.1,81.0,635,0.03,5.0,...,0.0,0.0,2.6,1.0,0.0,NaN,NaN,0.0,12.840920,1019.495876
